# CO project p1

# Initializing (preferances and capacities)

In [1]:
#Randomly

import numpy as np
import random
#cap of profs

#setting the seed
np.random.seed(10)
random.seed(10)


cap = np.random.normal(loc=1.2, scale=1.0, size=(1,12))
cap = np.floor(cap)
for i in range(len(cap[0])):
    if cap[0][i]<1:
        cap[0][i]=1
cap= cap.astype(int)
f = 30- np.sum(cap)
print(cap, "cap")
t = np.random.choice([i for i in range(0,12)], size=f)
print(t, "t")
for i in t:
    cap[0][i]+= 1
print(cap, "cap1")


studs = [i for i in range(1,31)]
profs = [i for i in range(1,13)]
studs_copy = studs.copy()
profs_copy = profs.copy()

def shuffle(a):
    np.random.shuffle(a)
    return a

pref_studs = np.array([shuffle(profs.copy()) for stud in studs])
pref_profs = np.array([shuffle(studs.copy()) for prof in profs])

print(np.shape(pref_studs))
print(np.shape(pref_profs))

[[2 1 1 1 1 1 1 1 1 1 1 2]] cap
[ 8  4  1  3  6  5  3  9  6  9  1  9  4 11  2  6] t
[[2 3 2 3 3 2 4 1 2 4 1 3]] cap1
(30, 12)
(12, 30)


In [2]:
#Better, more chance to be preffered

ss = (30*31)/2
ps = (12*13)/2
P_stud = [i/ss for i in range(1,31)]
P_prof = [i/ps for i in range(1,13)]

def choice(a,P):
    t = np.random.choice(a, size=len(a), replace=False, p=P)
    return t

pref_studs_ = np.array([choice(profs.copy(), P_prof) for stud in studs])
pref_profs_ = np.array([choice(studs.copy(), P_stud) for prof in profs])

print(np.shape(pref_studs_))
print(np.shape(pref_profs_))

(30, 12)
(12, 30)


In [3]:
print(pref_profs_[1])

[23 20 14 28 16 30 18 29 13 21 27 19 24 17  9 12 25  3  2 26 10 15 22  8
  4 11  6  5  7  1]


# Model

def model

In [4]:
import pyomo.environ as pyo 

In [5]:
model=pyo.ConcreteModel()

Sets and Params

In [6]:
model.iset= pyo.Set(initialize= profs)
model.jset= pyo.Set(initialize= studs)

In [7]:
print(model.iset.data())
print(model.jset.data())

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)
(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30)


In [8]:
def rule_cap(model, i):
    return cap[0][i-1]
model.cap = pyo.Param(model.iset, rule=rule_cap)

In [9]:
def rule_pref_studs(model,i,j):
    return pref_studs[j-1,i-1]
model.pref_studs= pyo.Param(model.iset, model.jset, rule=rule_pref_studs)

In [10]:
def rule_pref_profs(model,i,j):
    return pref_profs[i-1,j-1]
model.pref_profs= pyo.Param(model.iset, model.jset, rule=rule_pref_profs)

In [11]:
def rule_pref_studs_(model,i,j):
    return pref_studs_[j-1,i-1]
model.pref_studs_= pyo.Param(model.iset, model.jset, rule=rule_pref_studs_)

In [12]:
def rule_pref_profs_(model,i,j):
    return pref_profs_[i-1,j-1]
model.pref_profs_= pyo.Param(model.iset, model.jset, rule=rule_pref_profs_)

In [13]:
def rule_weights(model,i,j):
    return pref_profs[i-1,j-1]*pref_studs[j-1,i-1]
model.weights= pyo.Param(model.iset, model.jset, rule=rule_weights)

In [14]:
def rule_weights_(model,i,j):
    return pref_profs_[i-1,j-1]*pref_studs_[j-1,i-1]
model.weights_= pyo.Param(model.iset, model.jset, rule=rule_weights_)

In [15]:
print(pyo.value(model.weights_[1,1]))

270


def Variables

In [16]:
model.delta=pyo.Var(model.iset, model.jset, domain=pyo.Binary)

def Constraints

In [17]:
def const_profs(model,i):
    return sum(model.delta[i,j] for j in model.jset)== model.cap[i]
model.const_profs= pyo.Constraint(model.iset, rule= const_profs)

In [18]:
def const_studs(model,j):
    return sum(model.delta[i,j] for i in model.iset)== 1
model.const_profs= pyo.Constraint(model.iset, rule= const_studs)

    'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


def Objective

In [19]:
def obj(model):
    return sum(model.delta[i,j]*model.weights_[i,j]*(1/model.cap[i]) for i in model.iset for j in model.jset)

In [20]:
model.obj= pyo.Objective(rule=obj, sense=pyo.maximize)

# Solving and analysing the result

Solving

In [21]:
opt = pyo.SolverFactory('glpk')
result=opt.solve(model,'glpk')

In [22]:
a = sum(1/model.cap[i] for i in model.iset)
a = a/len(model.cap)
a = 1/a
print(a)

2.0571428571428574


In [23]:
w = pyo.value(model.obj)
w = w*a*(1/12)*(1/30)
print(np.sqrt(w))

7.845896713448071


In [24]:
w= np.sqrt(w)
w= w/30
print('model: ', w)

model:  0.261529890448269


In [25]:
e = sum(model.weights_[i,j]*(1/model.cap[i]) for i in model.iset for j in model.jset)
e = e*a*(1/12)*(1/30)
e= np.sqrt(e)
e= e/30
print(e)

0.33181160588343167


In [26]:
e= e/12
print('random: ',e)

random:  0.02765096715695264


# Therefore, the comparision shows that the optimization objective results in average happiness of professors and students, about 10 times more than paring them randomly, so it could be concluded that the objective function is proper. 




In [27]:
model.display()

Model unknown

  Variables:
    delta : Size=360, Index=delta_index
        Key      : Lower : Value : Upper : Fixed : Stale : Domain
          (1, 1) :     0 :   1.0 :     1 : False : False : Binary
          (1, 2) :     0 :   0.0 :     1 : False : False : Binary
          (1, 3) :     0 :   0.0 :     1 : False : False : Binary
          (1, 4) :     0 :   0.0 :     1 : False : False : Binary
          (1, 5) :     0 :   0.0 :     1 : False : False : Binary
          (1, 6) :     0 :   0.0 :     1 : False : False : Binary
          (1, 7) :     0 :   0.0 :     1 : False : False : Binary
          (1, 8) :     0 :   0.0 :     1 : False : False : Binary
          (1, 9) :     0 :   0.0 :     1 : False : False : Binary
         (1, 10) :     0 :   0.0 :     1 : False : False : Binary
         (1, 11) :     0 :   0.0 :     1 : False : False : Binary
         (1, 12) :     0 :   1.0 :     1 : False : False : Binary
         (1, 13) :     0 :   1.0 :     1 : False : False : Binary
        